In [132]:
import pandas as pd
import numpy as np
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract

In [133]:
from pytesseract import Output
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

- 경로지정

In [134]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd=R'C:\Program Files\Tesseract-OCR\tesseract'

In [135]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

- 정확도

In [136]:
def get_accuracy(answer_df, predict_df):
    return accuracy_score(answer_df['text'].values, predict_df['text'].values)

- PyTesseract Model

In [137]:
class PyTesseract:
    def __init__(self, lang='kor'):
        self.lang = lang
    
    #이미지 분석시 쉽게 Gray로 변환
    def load_image(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return img
    
    #텍스트 전처리
    def text_preprocessing(self, text):
        text = text.replace('\n', '')
        text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', text)
        text = text.strip()
        return text
    
    #
    def prediction(self, img_path_list):
        preds = []
        for img_path in tqdm(img_path_list):
            img = self.load_image(img_path)
            
            norm_img=np.zeros((img.shape[0], img.shape[1]))
            img=cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
            img=cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)[1]
            img=cv2.GaussianBlur(img, (1,1), 0)
            #img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            #img=cv2.GaussianBlur(img, ksize=(5,5), sigmaX=0)
            config = ('-l kor --oem 3 --psm 4')
            text = pytesseract.image_to_string(img, config=config)
            text = self.text_preprocessing(text)
            preds.append(text)
        print('Done.')
        return preds

In [138]:
tesseract_model = PyTesseract()

- tesseract_model로 train데이터 예측

In [130]:
train_predicts = tesseract_model.prediction(train_df['img_path'].values[20:30])

  0%|          | 0/10 [00:00<?, ?it/s]

Done.


In [131]:
train_predicts

['', '', '', '}110 조아', '', '', '', '', '', '0 바']

In [129]:
#train이미지 경로
train_predict_df=train_df.copy()
#train_predict_df=train_predicts
#print('Train Accuracy :', get_accuracy(train_df, train_predict_df))
train_predict_df

,img_path,text
0,./train/train_00001.png,골목미용실
1,./train/train_00002.png,한성부동산
2,./train/train_00003.png,홍라운지
3,./train/train_00004.png,모단걸응접실
4,./train/train_00005.png,씨앗양식
...,...,...
12154,./train/train_12161.png,넬슨
12155,./train/train_12162.png,앤드루 램버트 지음
12156,./train/train_12163.png,박아람 옮김
12157,./train/train_12164.png,판결의


In [139]:
test_predicts=tesseract_model.prediction(test_df['img_path'].values)

  0%|          | 0/3669 [00:00<?, ?it/s]

Done.


In [141]:
test_predicts

['',
 '',
 '26 6',
 '',
 '학교',
 '롬노인요양원',
 '한우리 건죽사사무소',
 '',
 '',
 '',
 '으',
 '피엘라벤 진주점',
 '',
 '',
 '',
 '',
 '다정',
 '미용',
 '',
 '시카고',
 '',
 '',
 '',
 '',
 '0 거그 는 뜨타66001  6',
 '실사현수막',
 '',
 '',
 '',
 '',
 '째흥르드',
 '',
 '제보 옥 쟁매주 전문서그 0 1  제노 ㅁ25',
 '9      자   그 여      뜨       개 그    이         느',
 '',
 '',
 '',
 '',
 '그 6고여“',
 '24 며',
 '',
 '',
 '',
 '배다귀탕',
 '배09사',
 '',
 '',
 '',
 '',
 '',
 '그니 노고 타베승인중개',
 '',
 '부기',
 '프라임 리빙텔',
 '셀프사진관',
 '',
 '',
 '',
 '스근보그 고는 존호 날조 둘로',
 '',
 '',
 '',
 '',
 '폐인트',
 '2727',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '계안안이',
 '',
 '',
 '',
 '',
 '용인소방서',
 '',
 '',
 '',
 '',
 '',
 '정지하이르3 ㄱㄱ ㄴㄴ  드',
 '만모스문구',
 '',
 '전기그이',
 '한량동',
 '',
 '뽀송뽀송',
 '',
 '다 ㅋ2 듬샘',
 'ㄷㄴ족서논술',
 '',
 '전용주   주차장',
 '뮤앰영어',
 '신동백 스크린끌프',
 '',
 '',
 '서2',
 '',
 '조ㄷ10주택청약',
 '',
 '이즈믹',
 '질시풍',
 '악세사리',
 '',
 '위험 고압가스',
 '',
 '',
 '',
 '',
 '각종요리',
 '',
 '',
 '',
 '',
 '',
 '본베이',
 '',
 '',
 '에 우으',
 '',
 '',
 '',
 '디마리',
 '',
 '',
 '7여1',
 '썼',
 ''

In [144]:
submit=pd.read_csv('sample_submission.csv')
submit['text']=test_predicts
submit.to_csv('submit.csv', index=False, encoding='utf-8-sig')